# <center> [Ноутбук для решения практики](https://stepik.org/lesson/1500755/step/12?unit=1520869)

# 1️⃣ **Описание шаблона для решения задачи.**

**Задача**: обучить CatBoost, залогировать основные компоненты
Вам необходимо сдать файл с расширением любое_имя.py в котором:

**Базовое задание (5 баллов)**

* Будет загрузка датасета
* Разделение на тренировочную и валидационную выборки
* Логирование только валидационной выборки
* Обучение бустинга с логированием процесса обучения в ClearML и сохранением гиперпараметров модели
* Расчет и сохранение метрики на валидационной выборке (classification report и Accuracy)
* Сохранение обученной модели
  
**Дополнительные задания (2 балла)**

* Добавить возможность считывания 2-х параметров при запуске файла на исполнение:
  + `-- iterations` - задаёт количество итераций бустинга (по умолчанию 500)
  + `-- verbose` - задаёт вывод прогресса обучения CatBoost в консоль (по умолчанию False)
  
Пример команды: `python любое_имя.py --iterations 200 --verbose 100`

* Провести EDA и сохранить графики в ClearML

👀 При желании, рекомендуется проделать следующее:
- Добавить теги для эксперимента
- Добавить еще метрик и отслеживать их по мере обучения (главное в меру 😁)


❗️❗️❗️ **P.S.** Данный ноутбук - далеко не единственное верное решение, воспринимайте его как помощник для вашего собственного решения или чтобы побороть страх белого листа :)

# 2️⃣ Подключаем необходимые библиотеки

In [ ]:
!pip install clearml catboost -q

In [9]:
from dataclasses import dataclass, asdict

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from catboost import CatBoostClassifier
from clearml import Logger, Task

In [11]:
# Вносим наши ключи в переменные среды
import os
from getpass import getpass


os.environ["CLEARML_API_ACCESS_KEY"] = getpass(prompt="Введите ваш access_key")
os.environ["CLEARML_API_SECRET_KEY"] = getpass(prompt="Введите ваш secret_key")

Введите ваш access_key ········
Введите ваш secret_key ········


<div class="alert alert-warning">

Необходимо получить access и secret токены https://app.clear.ml/settings/workspace-configuration

Если работаете на локальном компьютере или сервере можете ввести креды в консоли командой `clearml-init`.

In [3]:
%%capture
#  Не показывать свои api-ключи
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml

In [4]:
@dataclass
class CFG:
    project_name: str = "..."
    experiment_name: str = "..."

    data_path: str = "../data"
    train_name: str = "quickstart_train.csv"
    seed: int = 2024


cfg = CFG()

In [5]:
def seed_everything(seed=2024):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(cfg.seed)

# 3️⃣ Начинаем эксперимент

In [12]:
task = Task.init(project_name=cfg.project_name, task_name=cfg.experiment_name)
task.add_tags(["CB_classifier"])  # Добавьте тэги обучения

ClearML Task: created new task id=f41687231a4d41a1a3ed100fb5a8bd3c
2024-12-18 14:56:14,458 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/d8a11f463b1c42b78313f89b3c0a7a31/experiments/f41687231a4d41a1a3ed100fb5a8bd3c/output/log


In [ ]:
# Добавить конфиг запуска
task.connect(...)

# 4️⃣ Подгружаем данные

In [13]:
url = "https://github.com/a-milenkin/ml_instruments/raw/refs/heads/main/data/quickstart_train.csv"
rides_info = pd.read_csv(url)

## Препроцессинг

In [14]:
rides_info.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,32.22,engine_fuel,3.880920,1.651883e+07,0.1,172.793237,-5.029476,174,170
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,27.51,engine_fuel,4.181149,1.398317e+07,0.1,203.462289,-14.260456,174,171


In [15]:
cat_features = ["model", "car_type", "fuel_type"]  # Выделяем категориальные признаки
targets = ["target_class", "target_reg"]
features2drop = ["car_id"]  # эти фичи будут удалены

# Отбираем итоговый набор признаков для использования моделью
filtered_features = [
    i for i in rides_info.columns if (i not in targets and i not in features2drop)
]
num_features = [i for i in filtered_features if i not in cat_features]

print("cat_features", cat_features)
print("num_features", len(num_features))
print("targets", targets)

for c in cat_features:  # Избавлеямся от NaN'ов
    rides_info[c] = rides_info[c].astype(str)

cat_features ['model', 'car_type', 'fuel_type']
num_features 11
targets ['target_class', 'target_reg']


In [16]:
train, test = train_test_split(rides_info, test_size=0.2, random_state=cfg.seed)

In [ ]:
# Залогируйте только валидационную выборку!

logger.report_table(...)

In [17]:
X_train = train[filtered_features]
y_train = train["target_class"]

X_test = test[filtered_features]
y_test = test["target_class"]

# 5️⃣ Обучаем модельку

In [18]:
cb_params = {
    "depth": 4,
    "learning_rate": 0.06,
    "loss_function": "MultiClass",
    "custom_metric": ["Recall"],
    # Главная фишка катбуста - работа с категориальными признаками
    "cat_features": cat_features,
    # Регуляризация и ускорение
    "colsample_bylevel": 0.098,
    "subsample": 0.95,
    "l2_leaf_reg": 9,
    "min_data_in_leaf": 243,
    "max_bin": 187,
    "random_strength": 1,
    # Параметры ускорения
    "task_type": "CPU",
    "thread_count": -1,
    "bootstrap_type": "Bernoulli",
    # Важное!
    "random_seed": cfg.seed,
    "early_stopping_rounds": 50,
}

залогируйте параметры CatBoost

Логирование CatBoost в ClearML - https://clear.ml/docs/latest/docs/guides/frameworks/catboost/

In [19]:
model = CatBoostClassifier(**cb_params)

In [20]:
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

0:	learn: 2.1695430	test: 2.1730530	best: 2.1730530 (0)	total: 51.3ms	remaining: 51.2s
100:	learn: 1.0754529	test: 1.0957613	best: 1.0957613 (100)	total: 311ms	remaining: 2.77s
200:	learn: 0.7156387	test: 0.7521605	best: 0.7521605 (200)	total: 571ms	remaining: 2.27s
300:	learn: 0.5612275	test: 0.6139112	best: 0.6139112 (300)	total: 853ms	remaining: 1.98s
400:	learn: 0.5014757	test: 0.5659625	best: 0.5659625 (400)	total: 1.13s	remaining: 1.69s
500:	learn: 0.4642412	test: 0.5380338	best: 0.5379895 (496)	total: 1.4s	remaining: 1.39s
600:	learn: 0.4417683	test: 0.5242665	best: 0.5242665 (600)	total: 1.64s	remaining: 1.09s
700:	learn: 0.4238230	test: 0.5143201	best: 0.5142030 (699)	total: 1.9s	remaining: 812ms
800:	learn: 0.4123398	test: 0.5083812	best: 0.5083584 (798)	total: 2.17s	remaining: 539ms
900:	learn: 0.3985460	test: 0.5032266	best: 0.5031747 (899)	total: 2.44s	remaining: 268ms
999:	learn: 0.3867535	test: 0.4981496	best: 0.4980510 (997)	total: 2.73s	remaining: 0us

bestTest = 0.498

In [21]:
# сохраняем модель
model.save_model("cb_model.cbm")

# Метрики на тесте

In [25]:
y_pred = model.predict(X_test)

In [28]:
accuracy_score(y_test, y_pred)

0.8012820512820513

In [26]:
cls_report = classification_report(
    y_test, y_pred, target_names=y_test.unique(), output_dict=True
)

In [27]:
cls_report = pd.DataFrame(cls_report).T
cls_report

,precision,recall,f1-score,support
electro_bug,0.933333,0.933333,0.933333,60.000000
engine_ignition,1.000000,1.000000,1.000000,55.000000
another_bug,1.000000,1.000000,1.000000,52.000000
gear_stick,0.712121,0.903846,0.796610,52.000000
engine_overheat,0.756098,0.620000,0.681319,50.000000
engine_fuel,0.568182,0.462963,0.510204,54.000000
break_bug,0.485294,0.611111,0.540984,54.000000
engine_check,0.981132,0.912281,0.945455,57.000000
wheel_shake,0.827586,0.705882,0.761905,34.000000
accuracy,0.801282,0.801282,0.801282,0.801282


# 6️⃣ Сохраняем результаты в ClearML 

In [ ]:
# Логируем метрики

logger.

In [31]:
# Не забываем завершить таск
task.close()

# 7️⃣ Запись файла для сдачи на Stepik

In [ ]:
%%writefile example.py
#!/usr/bin/env python
from catboost import CatBoostClassifier
# your code here

def main():
    pass # your code here

if __name__ == "__main__":
    # your code here
    main()

**Проверка работоспособности**

In [ ]:
!python example.py

In [ ]:
# Для бонусного задания
!python example.py --iterations 200 --verbose 100

## 🎱 Поздравляем, вы стали на шаг ближе к продакшен-разработке ML-проектов!
Теперь сможете писать код для запуска экспериментов на удаленном сервере, даже при отсутствии установленного на нём Jupyter-lab.